# Chapter 6: The ReAct Pattern - Multi-Step Reasoning

**ReAct** stands for **Reason + Act**. It's a pattern where the agent alternates between thinking and doing, following a **Think -> Act -> Observe** loop until it has enough information to answer the question.

In the last chapter, we built an agent that calls tools. Ask a question, call a tool, get an answer. Simple. But real finance questions aren't simple:

> "Compare Apple and NVIDIA's stock performance. Which one is trading closer to its 52-week high?"

To answer this, you need to:
1. Get Apple's current price
2. Get Apple's 52-week high
3. Get NVIDIA's current price
4. Get NVIDIA's 52-week high
5. Calculate the percentage for each
6. Compare and explain

That's not one tool call. That's a **chain of reasoning with multiple steps**.

Unlike Chapter 5's simulated data (hardcoded dictionaries), the tools in this notebook use **yfinance** to fetch **live market data**. Same agent pattern, real prices.

In [1]:
!pip install smolagents yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 12.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [2]:
from smolagents import CodeAgent, tool
from smolagents import OpenAIServerModel
from smolagents.monitoring import LogLevel

In [3]:
import getpass
API_KEY = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [4]:
model = OpenAIServerModel("gpt-4o-mini", api_key=API_KEY)
print("Model initialized!")

Model initialized!


## Setting Up the Tools

We have three tools for stock analysis. Each does one thing well --- same structure as before (same docstrings, same type hints) --- but now backed by **real market data** via `yfinance`.

Unlike Chapter 5's simulated data (hardcoded dictionaries with fixed prices), these tools call the Yahoo Finance API to get live prices. The book explicitly states this upgrade: *"Unlike Chapter 5's simulated data, these use yfinance to fetch live market data."*

The agent will combine them through the ReAct loop to answer complex questions.

In [5]:
@tool
def get_stock_price(ticker: str) -> float:
    """Get the current price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The current stock price as a float
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    return stock.info.get('regularMarketPrice', 0.0)

@tool
def get_52_week_high(ticker: str) -> float:
    """Get the 52-week high price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The 52-week high price as a float
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    return stock.info.get('fiftyTwoWeekHigh', 0.0)

@tool
def get_52_week_low(ticker: str) -> float:
    """Get the 52-week low price for a stock ticker.

    Args:
        ticker: The stock symbol (e.g., 'AAPL', 'NVDA', 'MSFT')

    Returns:
        The 52-week low price as a float
    """
    import yfinance as yf
    stock = yf.Ticker(ticker)
    return stock.info.get('fiftyTwoWeekLow', 0.0)

print("Three tools created: get_stock_price, get_52_week_high, get_52_week_low")

Three tools created: get_stock_price, get_52_week_high, get_52_week_low


## The Multi-Step Query

Now let's ask a complex question and watch the ReAct loop in action.

A question like *"Compare Apple and NVIDIA --- which is closer to its 52-week high?"* triggers multiple Think -> Act -> Observe cycles. The agent:
1. **Plans** its approach (what data do I need?)
2. **Calls tools** one by one (get each piece of data)
3. **Synthesizes** the results into a final answer

We use `verbosity_level=LogLevel.INFO` so you can see the reasoning at each step.

In [6]:
agent = CodeAgent(
    tools=[get_stock_price, get_52_week_high, get_52_week_low],
    model=model,
    verbosity_level=LogLevel.INFO
)

In [7]:
result = agent.run("""
Compare Apple and NVIDIA:
- What's each stock's current price?
- What's each stock's 52-week high?
- Which stock is trading closer to its 52-week high (as a percentage)?
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare Apple and NVIDIA:                                                                                       │
│ - What's each stock's current price?                                                                            │
│ - What's each stock's 52-week high?                                                                             │
│ - Which stock is trading closer to its 52-week high (as a percentage)?                                          │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Getting the current prices for both Apple and NVIDIA                                                           
  apple_price = get_stock_price(ticker='AAPL')                                                                     
  nvidia_price = get_stock_price(ticker='NVDA')                                                                    
                                                                                                                   
  # Getting the 52-week high prices for both Apple and NVIDIA                                                      
  apple_52_week_high = get_52_week_high(ticker='AAPL')                                                             
  nvidia_52_week_high = get_52_week_high(ticker='NVDA')                                                            
                                                                                                                   
  # Printing all the values retrieved                                                                              
  print(f"Apple current price: {apple_price}, 52-week high: {apple_52_week_high}")                                 
  print(f"NVIDIA current price: {nvidia_price}, 52-week high: {nvidia_52_week_high}")                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Apple current price: 263.88, 52-week high: 288.62
NVIDIA current price: 184.97, 52-week high: 212.19

Out: None

[Step 1: Duration 9.88 seconds| Input tokens: 2,169 | Output tokens: 302]

...

[Step 2: Duration 8.03 seconds| Input tokens: 4,887 | Output tokens: 649]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "apple_current_price": 263.88,                                                                               
      "apple_52_week_high": 288.62,                                                                                
      "apple_percentage": apple_percentage,                                                                        
      "nvidia_current_price": 184.97,                                                                              
      "nvidia_52_week_high": 212.19,                                                                               
      "nvidia_percentage": nvidia_percentage,                                                                      
      "closer_stock": closer_stock                                                                                 
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'apple_current_price': 263.88, 'apple_52_week_high': 288.62, 'apple_percentage': 91.42817545561638, 
'nvidia_current_price': 184.97, 'nvidia_52_week_high': 212.19, 'nvidia_percentage': 87.17187426363165, 
'closer_stock': 'Apple'}

[Step 3: Duration 4.85 seconds| Input tokens: 8,254 | Output tokens: 818]

## What Just Happened?

The agent followed the ReAct loop --- multiple cycles of Think, Act, Observe:

1. **THINK:** "I need Apple's current price" -> **ACT:** `get_stock_price("AAPL")` -> **OBSERVE:** the result
2. **THINK:** "Now I need Apple's 52-week high" -> **ACT:** `get_52_week_high("AAPL")` -> **OBSERVE:** the result
3. **THINK:** "Got Apple's data. Now NVIDIA's price" -> **ACT:** `get_stock_price("NVDA")` -> **OBSERVE:** the result
4. **THINK:** "Now NVIDIA's 52-week high" -> **ACT:** `get_52_week_high("NVDA")` -> **OBSERVE:** the result
5. **THINK:** "I have all the data. Let me calculate percentages and compare" -> **FINAL ANSWER**

Each THINK step decides what to do next based on what it's already learned. The agent isn't following a script --- it's **reasoning through the problem dynamically**.

> **Note:** LLM outputs are non-deterministic. Your results may differ in wording, step order, and actual values (these are live market prices). The *pattern* will be the same.

## Graceful Handling of Edge Cases

What happens when the agent encounters bad data or unknown tickers? The ReAct pattern handles this naturally through its Observe step.

When `yfinance` returns 0.0 for an unknown ticker, the agent **reasons** about what that means rather than blindly reporting "the range is 0 to 0." It recognizes that zero indicates missing data and communicates that explicitly.

This is the reasoning component at work --- the agent doesn't just relay numbers, it interprets them.

In [9]:
result = agent.run("What's the 52-week range for PESLA? Not TSLA.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the 52-week range for PESLA? Not TSLA.                                                                   │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pesla_52_week_high = get_52_week_high("PESLA")                                                                   
  pesla_52_week_low = get_52_week_low("PESLA")                                                                     
  52_week_range = (pesla_52_week_low, pesla_52_week_high)                                                          
  print("52-week range for PESLA:", 52_week_range)                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code parsing failed on line 3 due to: SyntaxError: invalid decimal literal (<unknown>, line 3)
52_week_range = (pesla_52_week_low, pesla_52_week_high)   ^

[Step 1: Duration 3.85 seconds| Input tokens: 2,143 | Output tokens: 151]

...

[Step 4: Duration 4.29 seconds| Input tokens: 10,413 | Output tokens: 562]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The 52-week range for PESLA is not available.")                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The 52-week range for PESLA is not available.

[Step 5: Duration 3.00 seconds| Input tokens: 13,727 | Output tokens: 678]

## A More Complex Example

This query requires comparing three stocks and making a judgment call based on the data. The agent needs to:
1. Fetch the current price and 52-week high for all three stocks
2. Calculate how far each is from its high
3. Determine which has the most "upside" potential
4. Add appropriate caveats (this is a simplified analysis)

The agent doesn't just return numbers. It answers the **actual question**: which has more potential upside? That's the **reasoning** part of ReAct.

In [10]:
result = agent.run("""
I'm thinking about buying one of these stocks.
Which one is furthest from its 52-week high,
suggesting more potential upside?
Compare AAPL, NVDA, and MSFT.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm thinking about buying one of these stocks.                                                                  │
│ Which one is furthest from its 52-week high,                                                                    │
│ suggesting more potential upside?                                                                               │
│ Compare AAPL, NVDA, and MSFT.                                                                                   │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Get current prices and 52-week highs for AAPL, NVDA, MSFT                                              
  tickers = ["AAPL", "NVDA", "MSFT"]                                                                               
  current_prices = {}                                                                                              
  week_highs = {}                                                                                                  
                                                                                                                   
  for ticker in tickers:                                                                                           
      current_prices[ticker] = get_stock_price(ticker)                                                             
      week_highs[ticker] = get_52_week_high(ticker)                                                                
                                                                                                                   
  print("Current Prices:", current_prices)                                                                         
  print("52-Week Highs:", week_highs)                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Current Prices: {'AAPL': 263.88, 'NVDA': 184.97, 'MSFT': 396.86}
52-Week Highs: {'AAPL': 288.62, 'NVDA': 212.19, 'MSFT': 555.45}

Out: None

[Step 1: Duration 6.59 seconds| Input tokens: 2,167 | Output tokens: 282]

...

[Step 2: Duration 5.20 seconds| Input tokens: 4,845 | Output tokens: 503]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 4: Round the potential upside for final answer                                                            
  rounded_upside = round(furthest_difference, 2)                                                                   
  final_answer(f"The stock furthest from its 52-week high is {furthest_stock} with a potential upside of           
  ${rounded_upside}.")                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: The stock furthest from its 52-week high is MSFT with a potential upside of $158.59.

[Step 3: Duration 3.35 seconds| Input tokens: 8,006 | Output tokens: 617]

## Memory Across ReAct Steps

ReAct works naturally with memory. After the agent completes a multi-step analysis, you can ask **follow-up questions** that build on the previous results.

The key is `reset=False`. By default, each `agent.run()` call starts fresh --- the agent forgets everything from the last query. With `reset=False`, the agent **preserves its memory** from previous runs, enabling multi-turn analysis.

This is powerful for iterative exploration:
- First query: "Compare AAPL and NVDA"
- Follow-up: "Now add MSFT to that comparison"

The agent remembers the previous analysis and extends it rather than starting over.

In [11]:
# First query: full analysis
result = agent.run("""
Compare AAPL and NVDA: which is closer to its 52-week high?
""")

# Follow-up: build on previous analysis
result = agent.run("""
Now add MSFT to that comparison.
""", reset=False)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare AAPL and NVDA: which is closer to its 52-week high?                                                     │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get the current prices for AAPL and NVDA                                                                       
  aapl_price = get_stock_price(ticker='AAPL')                                                                      
  nvda_price = get_stock_price(ticker='NVDA')                                                                      
                                                                                                                   
  # Get the 52-week highs for AAPL and NVDA                                                                        
  aapl_52_week_high = get_52_week_high(ticker='AAPL')                                                              
  nvda_52_week_high = get_52_week_high(ticker='NVDA')                                                              
                                                                                                                   
  # Calculate how close each stock price is to its 52-week high                                                    
  aapl_difference = aapl_52_week_high - aapl_price                                                                 
  nvda_difference = nvda_52_week_high - nvda_price                                                                 
                                                                                                                   
  # Determine which is closer to its 52-week high                                                                  
  aapl_closeness = aapl_difference / aapl_52_week_high                                                             
  nvda_closeness = nvda_difference / nvda_52_week_high                                                             
                                                                                                                   
  print(f"AAPL closeness to 52-week high: {aapl_closeness}")                                                       
  print(f"NVDA closeness to 52-week high: {nvda_closeness}")                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
AAPL closeness to 52-week high: 0.08571824544383622
NVDA closeness to 52-week high: 0.12828125736368348

Out: None

[Step 1: Duration 7.99 seconds| Input tokens: 2,146 | Output tokens: 329]

...

[Step 3: Duration 8.09 seconds| Input tokens: 7,976 | Output tokens: 793]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("AAPL is closest to its 52-week high compared to NVDA and MSFT.")                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: AAPL is closest to its 52-week high compared to NVDA and MSFT.

[Step 4: Duration 2.75 seconds| Input tokens: 11,724 | Output tokens: 933]

## Tracking Steps Programmatically

You can access the agent's memory to see exactly what tools were called and in what order. This creates an **audit trail** --- essential in finance where you need to explain and justify every decision.

The `agent.memory.steps` list contains every step the agent took. Each step has a type (e.g., `ActionStep`, `TaskStep`) and may contain `tool_calls` showing exactly which tools were invoked with which arguments.

This programmatic access is useful for:
- Building audit logs
- Debugging unexpected behavior
- Extracting tool call sequences for analysis

In [12]:
# Run a query
result = agent.run("Compare the 52-week ranges of AAPL and MSFT")

# Access memory to see all steps
print("AGENT MEMORY:")
for i, step in enumerate(agent.memory.steps):
    step_type = type(step).__name__
    print(f"\nStep {i+1}: {step_type}")

    if hasattr(step, 'tool_calls') and step.tool_calls:
        for tc in step.tool_calls:
            print(f"   Tool: {tc.name}({tc.arguments})")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Compare the 52-week ranges of AAPL and MSFT                                                                     │
│                                                                                                                 │
╰─ OpenAIModel - gpt-4o-mini ─────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  aapl_high = get_52_week_high(ticker='AAPL')                                                                      
  aapl_low = get_52_week_low(ticker='AAPL')                                                                        
  msft_high = get_52_week_high(ticker='MSFT')                                                                      
  msft_low = get_52_week_low(ticker='MSFT')                                                                        
                                                                                                                   
  print("AAPL 52-week high:", aapl_high)                                                                           
  print("AAPL 52-week low:", aapl_low)                                                                             
  print("MSFT 52-week high:", msft_high)                                                                           
  print("MSFT 52-week low:", msft_low)                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
AAPL 52-week high: 288.62
AAPL 52-week low: 169.21
MSFT 52-week high: 555.45
MSFT 52-week low: 344.79

Out: None

[Step 1: Duration 4.95 seconds| Input tokens: 2,141 | Output tokens: 187]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "AAPL_52_week_high": aapl_high,                                                                              
      "AAPL_52_week_low": aapl_low,                                                                                
      "MSFT_52_week_high": msft_high,                                                                              
      "MSFT_52_week_low": msft_low,                                                                                
      "Comparison": "MSFT has a higher 52-week range than AAPL."                                                   
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'AAPL_52_week_high': 288.62, 'AAPL_52_week_low': 169.21, 'MSFT_52_week_high': 555.45, 
'MSFT_52_week_low': 344.79, 'Comparison': 'MSFT has a higher 52-week range than AAPL.'}

[Step 2: Duration 4.30 seconds| Input tokens: 4,699 | Output tokens: 377]

AGENT MEMORY:

Step 1: TaskStep

Step 2: ActionStep
   Tool: python_interpreter(aapl_high = get_52_week_high(ticker='AAPL')
aapl_low = get_52_week_low(ticker='AAPL')
msft_high = get_52_week_high(ticker='MSFT')
msft_low = get_52_week_low(ticker='MSFT')

print("AAPL 52-week high:", aapl_high)
print("AAPL 52-week low:", aapl_low)
print("MSFT 52-week high:", msft_high)
print("MSFT 52-week low:", msft_low))

Step 3: ActionStep
   Tool: python_interpreter(final_answer({
    "AAPL_52_week_high": aapl_high,
    "AAPL_52_week_low": aapl_low,
    "MSFT_52_week_high": msft_high,
    "MSFT_52_week_low": msft_low,
    "Comparison": "MSFT has a higher 52-week range than AAPL."
}))


## Exercise

Write a complex multi-step question that requires the agent to:
1. Fetch data for at least **2 stocks**
2. Perform some kind of **comparison or calculation**
3. Provide a **recommendation or conclusion**

**Ideas:**
- "Which stock has the widest 52-week range (high minus low)?"
- "If I had $10,000 to invest, how many shares of each could I buy?"
- "Which stock is most volatile based on its 52-week range as a percentage of its low?"

Run it with verbose mode and study the trace. Can you predict which tools the agent will call before it calls them?

In [ ]:
# YOUR MULTI-STEP QUESTION HERE
result = agent.run("""
    YOUR PROMPT HERE
""")